In [1]:
def is_clique(graph, nodes):    
    #Si solo hay un nodo, es un clique
    if len(nodes) == 1:
        return True
    
    #Para cada nodo de la lista miramos sus destinos
    #Para cada uno de los otros nodos de la lista, comprobamos que
    #se encuentra en la lista de destinos del primer nodo
    #Seguimos hasta que encontramos un nodo_destino que no esta en
    #la lista de destinos del nodo_origen o hasta que hemos recorrido
    #todos los nodos
    for nodo_origen in nodes:
        destinos = graph[nodo_origen]
        for nodo_destino in nodes:
            if nodo_origen != nodo_destino:
                if nodo_destino not in destinos:
                    return False
    
    return True

In [2]:
def is_coverage(graph, nodes):
    """
    Devuelve un booleano indicando si los nodos son un cubrimiento del grafo.
    Un conjunto de nodos es un cubrimiento si todos los arcos son adyacentes a
    al menos un nodo del conjunto.
    """    
    #Si no hay nodos, no esta cubierto
    if len(nodes)==0:
        return False
    
    #Creamos un conjunto con todos los arcos del nodo
    arcos_grafo = set([])
    for nodo_origen in graph:
        for nodo_destino in graph[nodo_origen]:
            arcos_grafo.add((nodo_origen, nodo_destino))

    #Creamos un conjunto con todos los arcos de los nodos
    arcos_nodo = set([])
    for nodo_origen in nodes:
        for nodo_destino in graph[nodo_origen]:            
            arcos_nodo.add((nodo_origen, nodo_destino))
            arcos_nodo.add((nodo_destino, nodo_origen))
            
    #Si la longitud de ambos conjuntos coincide, es un cubrimiento
    return len(arcos_grafo)==len(arcos_nodo)


In [6]:
def maxium_clique(graph):
    resultado = []
    nodos = set(graph.keys())    
    
    if len(nodos) <= 1:
        resultado.append(nodos)
        return resultado
    
    resultado = clique_recursivo(graph, nodos)
        
    return max(resultado, key=len)

#Meteremos los nodos en un set y iremos procesando uno a uno
#Llamaremos recursivamente a la funcion con los nodos relacionados
#con el nodo que estamos procesando 
#Iremos ampliando el resultado final con los nodos devueltos por las
#llamadas recursivas
#Y para eliminar iteraciones imnecesarias, quitaremos de la lista de
#pendientes los nodos que ya hayamos añadido por las llamadas recursivas
def clique_recursivo(grafo, nodos):
    """Dado un grafo, devuelve un conjunto de nodos que es un clique máximo."""

    resultado = []
    
    if len(nodos) <= 1:
        resultado.append(nodos)
        return resultado
    
    pendientes = set(nodos)
    while len(pendientes) != 0:
        nodo = pendientes.pop()
        nuevo_grafo = clique_recursivo(grafo, grafo[nodo] & nodos)
        for nuevo_nodo in nuevo_grafo:
            resultado.append(set([nodo]) | nuevo_nodo )
            pendientes = pendientes - nuevo_nodo
        
    return resultado

In [8]:
def minimum_coverage(graph):
    arcos_cubiertos = set([])    
    lista_nodos = []    
    nodos = []
    
    #Creamos un conjunto con todos los arcos del grafo
    arcos_grafo = set([])
    for nodo_origen in graph:
        lista_nodos.append(nodo_origen)
        for nodo_destino in graph[nodo_origen]:
            arcos_grafo.add((nodo_origen, nodo_destino))

    
    hay_cubrimiento = False
    while not hay_cubrimiento:
        #Dada la situacion actual, miramos cual es el mejor nodo que podemos
        #añadir y que nos aporta        
        mejoria = mejor_adicion(graph, lista_nodos, arcos_cubiertos)        
        if mejoria[0] != 0:
            #Quitamos el nodo de la lista de nodos pendientes, lo metemos en 
            #la lista de nodos resultado e incluimos sus arcos como cubiertos
            lista_nodos.remove(mejoria[0])
            nodos.append(mejoria[0])                        
            arcos_cubiertos = mejoria[2]
        else:
            #Ningun nodo nos aporta nada, no podemos completar el cubrimiento
            return nodos            
                
        if len(arcos_cubiertos) == len(arcos_grafo):            
            hay_cubrimiento = True            
            break
       
        if len(lista_nodos)==0:
            #No nos quedan mas nodos en la lista y no tenemos cubrimiento, salimos
            return nodos            
    
    return nodos

def mejor_adicion(grafo, lista_nodos, arcos_cubiertos):
    #Para cada nodo de la lista, vamos a ver que arcos tiene
    #y cuantos podriamos añadir a nuestro conjunto de arcos
    #El que mas arcos nos aporte, sera el que seleccionemos
    mejor = (0, 0, arcos_cubiertos)    
    for nodo in lista_nodos:        
        arcos_temporales = set([])
        arcos_nodo = []
        for nodo_destino in grafo[nodo]:
            arcos_nodo.append((nodo, nodo_destino))
            arcos_nodo.append((nodo_destino, nodo))
            
        arcos_temporales = arcos_cubiertos.union( set(arcos_nodo) )        
        mejoria = len(arcos_temporales) - len(arcos_cubiertos)
        if (mejoria > mejor[1]):
            mejor = (nodo, mejoria, arcos_temporales)
    
    return mejor